In [2]:
import numpy as np

class PolynomialRegression:
    def __init__(self, degree=2, learning_rate=0.01, iterations=1000):
        self.degree = degree          # Polynomial degree
        self.lr = learning_rate       # Learning rate
        self.iterations = iterations  # Gradient descent iterations
        self.weights = None           # Model parameters (including bias)
        self.mean = None              # For feature scaling
        self.std = None               # For feature scaling
        
    def _create_poly_features(self, X):
        """Generate polynomial features up to specified degree"""
        n_samples = X.shape[0]
        poly_features = np.ones((n_samples, self.degree + 1))
        
        for d in range(1, self.degree + 1):
            poly_features[:, d] = X[:, 0] ** d
            
        return poly_features
    
    def _scale_features(self, X):
        """Standardize features (except the first column of ones)"""
        X_scaled = X.copy()
        if self.degree > 1:  # Only scale if polynomial degree > 1
            # Skip first column (bias term)
            X_scaled[:, 1:] = (X[:, 1:] - self.mean) / self.std
        return X_scaled
    
    def fit(self, X, y):
        """Train the polynomial regression model"""
        # Convert to numpy arrays and ensure proper shape
        X = np.array(X).reshape(-1, 1)
        y = np.array(y).reshape(-1)
        
        # 1. Create polynomial features
        X_poly = self._create_poly_features(X)
        
        # 2. Store scaling parameters (mean/std) for each feature
        if self.degree > 1:
            self.mean = np.mean(X_poly[:, 1:], axis=0)
            self.std = np.std(X_poly[:, 1:], axis=0)
            
        # 3. Scale features
        X_scaled = self._scale_features(X_poly)
        
        # 4. Initialize weights
        n_features = X_scaled.shape[1]
        self.weights = np.zeros(n_features)
        
        # 5. Gradient Descent
        for _ in range(self.iterations):
            # Calculate predictions
            y_pred = X_scaled @ self.weights
            
            # Calculate gradients
            error = y_pred - y
            gradients = (1 / len(y)) * X_scaled.T @ error
            
            # Update weights
            self.weights -= self.lr * gradients
            
        return self
    
    def predict(self, X):
        """Make predictions using trained model"""
        X = np.array(X).reshape(-1, 1)
        
        # 1. Create polynomial features
        X_poly = self._create_poly_features(X)
        
        # 2. Scale features using training parameters
        X_scaled = self._scale_features(X_poly)
        
        # 3. Make prediction
        return X_scaled @ self.weights
    
    def score(self, X, y):
        """Calculate R-squared score"""
        y_pred = self.predict(X)
        ss_res = np.sum((y - y_pred) ** 2)
        ss_tot = np.sum((y - np.mean(y)) ** 2)
        return 1 - (ss_res / ss_tot)

In [3]:
# Example usage
if __name__ == "__main__":
    # Generate sample quadratic data: y = 2x² + 3x + 5 + noise
    np.random.seed(0)
    X = 2 * np.random.rand(100, 1)
    y = 2 * X**2 + 3 * X + 5 + np.random.randn(100, 1)
    y = y.ravel()  # Flatten to 1D array

    # Create and train model
    model = PolynomialRegression(degree=2, learning_rate=0.01, iterations=5000)
    model.fit(X, y)

    # Make predictions
    X_test = np.array([[0], [1], [2]])
    predictions = model.predict(X_test)

    # Print results
    print("Polynomial Regression Results (degree=2)")
    print("========================================")
    print(f"Weights: {model.weights}")
    print("\nTest predictions:")
    for x, pred in zip(X_test, predictions):
        print(f"Input {x[0]:.2f} -> Predicted y: {pred:.2f}")
    print(f"\nR-squared score: {model.score(X, y):.4f}")

Polynomial Regression Results (degree=2)
Weights: [10.48231202  2.17816403  1.82320198]

Test predictions:
Input 0.00 -> Predicted y: 4.97
Input 1.00 -> Predicted y: 10.33
Input 2.00 -> Predicted y: 18.85

R-squared score: 0.9417
